#### Arnoldi iteration

Previously we see that Arnoldi iteration can produce

$$AQ_n=Q_{n+1}H_{n+1, n}$$

and

$$H_n = Q_n^TAQ_n$$

with $H_n$ in upper Hessenberg form

In addition, we have

$$\text{span}(q_1, q_2, \cdots, q_n)=K_n(A, b)=\text{span}(b, Ab, \cdots, A^{n-1}b)$$

that is, the process iteratively generates `orthonormal basis` for Krylov subspace $K_n(A, b)$

#### Solving linear system of equations

One application of the Arnoldi iteration, other than helping to compute eigenvalues, is to solve linear system of equations $Ax=b$, where $A\in \mathbf{R}^{m \times m}$ is `full rank`

The idea is that at each iteration, we seek an approximate solution within the `Krylov subspace`

For example, at `nth` iteration, we let $x^n\in K_n(A,b)$, where $K_n(A, b)=\text{span}(b, Ab, \cdots, A^{n-1}b)$

We can put $b, Ab, \cdots, A^{n-1}b$ into a matrix $K_n$

$$K_n=\begin{bmatrix}b& Ab& \cdots& A^{n-1}b\end{bmatrix}$$

and the idea of `generalized minimal residuals` (GMRES) method is to solve the least squares problem for coefficients $c$ of the expansion of $x^n$ in the columns of $K_n$

$$\min_c \|AK_nc-b\|$$

Once we get $c$, we compute $x^n=K_nc$

While this representation is correct, in practice, it's more efficient to work with an orthonormal basis

With Arnoldi iteration, we know that $Q_n$ is the orthonormal basis for $K_n$ and since $AQ_n=Q_{n+1}H_{n+1,n}$, we can rewrite the optimization as

$$\begin{align*}
&\min_c \|AK_nc-b\|\\
& \text{use y as coefficients for expansion of x in } Q_n  \\
=&\min_y \|AQ_ny-b\| \\
=&\min_y \|Q_{n+1}H_{n+1,n}y-b\| \\
& \text{multiplying orthogonal matrix does not change norm}  \\
=&\min_y \|H_{n+1,n}y-Q_{n+1}^Tb\| \\
& \text{Arnoldi uses normalized b as first q by design}  \\
=&\min_y \|H_{n+1,n}y-\|b\|e_1\| \\
\end{align*}$$

Once we get $y$, we compute $x^n$ as

$$x^n=Q_ny$$

We repeat until the residual $\|H_{n+1,n}y-\|b\|e_1\|$ is below certain threshold

#### Convergence of GMRES

It can be shown that what GMRES does is to find $p\in P_n$, where $P_n$ is a set of all polynomials of degree $\leq n$ with $p(0)=1$, such that

$$\|p(A)b\|$$

is minimized

`Convergence theorem` from NLA book

Suppose $A$ is diagonalizable $A=V\Lambda V^{-1}$, at step $n$ of GMRES, the residual $r_n$ satisfies

$$\frac{\|r_n\|}{\|b\|}\leq \kappa(V) \inf_{p\in P_n}\|p\|_{\Lambda(A)}$$

where $\Lambda(A)$ is set of eigenvalues of $A$

$\|p\|_{\Lambda(A)}$ is defined as

$$\|p\|_{\Lambda(A)}=\sup_{z\in \Lambda(A)} |p(z)|$$

This expression means we're taking the supremum of the absolute value of the polynomial $p(z)$ evaluated at each eigenvalue $z$ of $A$. Essentially, it's the largest value that $|p(z)|$ attains over all eigenvalues of $A$

##### Interpretation

* We want a polynomial $p$, such that when we evaluate it at `any eigenvalue` of $A$, the absolute value $|p(z)|$ is `as small as possible`, that is, the polynomial can `uniformly` approximate zero over all eigevalues
* Further, if $V$ is well-conditioned, the bound is tighter, leading to potentially faster convergence

##### Judging matrix suitability for GMRES

* We prefer `clustered` eigenvalues, which is easier to find a polynomial that is small over all of them
* Eigenvalues `close to zero`, if eigenvalues are widely spread out, higher-degree polynomials may be needed, slowing down convergence
* Smaller $\kappa(V)$ implies $A$ is closer to `diagonalizable with an orthogonal matrix`, since orthogonal matrix has the smallest condition number of one

For example, symmetric matrices with orthogonal $V$ and eigenvalues that are relatively close and small, GMRES should converge well

##### Challenges

For general matrices, it is difficult to estimate $\kappa(V)$

In addition, while power iteration can be used to estimate the largest eigenvalue of $A$, finding the smallest can be challenging for large matrices, a few Arnoldi/Lanczos iterations may provide some idea regarding the distribution of eigenvalues

#### Example

From NLA book, there is an example of ideal case

* Eigenvalues clustered near one by using identity matrix plus some small perturbation
* Well-conditioned $V$ since the matrix is relatively close to identity matrix

In [18]:
import matplotlib.pyplot as plt
import numpy as np
import time

np.set_printoptions(formatter={'float': '{: 0.4f}'.format})

plt.style.use('dark_background')
# color: https://matplotlib.org/stable/gallery/color/named_colors.htm

In [19]:
def general_gram_schmidt(A, modified=True):
    _, k = A.shape  # Get number of vectors (columns) in A
    Q = []  # Start with empty list, as we don't know how many q's are there
    R = np.zeros((0, k))  # Same here

    for i in range(k):
        # Loop over all a_i
        q = A[:, i].copy()

        # This skips when i=0
        for j in range(len(Q)):
            if modified:
                R[j, i] = np.dot(Q[j], q)
            else:
                R[j, i] = np.dot(Q[j], A[:, i])
            q -= R[j, i] * Q[j]

        # Compute norm of new q
        norm_q = np.sqrt(np.dot(q, q))

        # Only add q to Q if it is not small
        if norm_q > 1e-10:  # Tolerance
            q /= norm_q
            Q.append(q)

            # Expand R to include new row corresponding to new q
            new_row = np.zeros((1, k))
            new_row[0, i] = norm_q
            R = np.vstack([R, new_row])

    Q = np.column_stack(Q)  # Convert to array

    return Q, R

def back_substitution(R, b):
    m, n = R.shape
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (b[i] - np.dot(R[i, i + 1:], x[i + 1:])) / R[i, i]
    return x

In [20]:
def gmres(A, b, tol=1e-10, n_restart=50, n_max=20):
    m, n_ = A.shape
    x = np.zeros(n_)
    r = b - A @ x
    residuals = [np.linalg.norm(r)]
    iter_count = 0
    n = min(n_, n_max)

    for k in range(n_restart):
        r_norm = np.linalg.norm(r)
        Q = np.zeros((m, n+1))
        H = np.zeros((n+1, n))
        Q[:, 0] = r / r_norm

        for i in range(n):
            iter_count += 1
            # Arnoldi iteration
            v = A @ Q[:, i]

            for j in range(i+1):
                H[j, i] = Q[:, j] @ v
                v -= H[j, i] * Q[:, j]

            H[i+1, i] = np.linalg.norm(v)
            if H[i+1, i] < tol:  # Early termination
                H = H[:i+2, :i+1]  # Truncate H
                Q = Q[:, :i+1]     # Truncate Q
                break
            Q[:, i+1] = v / H[i+1, i]

            # Least squares
            e1 = np.zeros(i+2) # Size equalling first dim of H_{n+1,n}, note i+2 since i starts at 0
            e1[0] = r_norm

            # H_{n+1,n} y = r_norm * e1
            # y, _, _, _ = np.linalg.lstsq(H[:i+2, :i+1], e1, rcond=None) # NumPy solver
            Q_ls, R_ls = general_gram_schmidt(H[:i+2, :i+1])
            y = back_substitution(R_ls, Q_ls.T @ e1)

            # Previous residual drop taken care of by x
            x_n = x + Q[:, :i+1] @ y # Current residual drop taken care of by Q[:, :i+1] @ y

            residual_norm = np.linalg.norm(b - A @ x_n)
            residuals.append(residual_norm)

            if residual_norm < tol:
                print(f"Converged at iteration {iter_count}")
                return x_n, residuals

        print(f"Iteration {iter_count}, residual norm: {residual_norm}")

        x = x_n
        r = b - A @ x # Fit only remaining residual for next restart

    print("Maximum iterations reached without convergence")
    return x, residuals

In [21]:
np.random.seed(42)

m = 3000
A = 1 * np.eye(m) + 0.5 * np.random.rand(m, m)/np.sqrt(m) # Ideal case
# A = np.random.rand(m, m) # Non-ideal case
print("Condition number:", np.linalg.cond(A))
x_true = np.random.rand(m)
b = A @ x_true
n_restart = 100
n_max = 1000

start_time = time.time()
x_approx, residuals = gmres(A, b, tol=1e-10, n_restart=n_restart, n_max = n_max)
print(f"Time taken: {time.time() - start_time} seconds")

print("x difference:", np.linalg.norm(x_true - x_approx))
print("b difference:", np.linalg.norm(b - A @ x_approx))

Condition number: 18.3223509474835
Converged at iteration 15
Time taken: 0.22419190406799316 seconds
x difference: 2.4002373922867235e-11
b difference: 2.3165023797534836e-11


#### Comparison with LU with partial pivoting

In [22]:
def lu_factorization(A):
    m, n = A.shape
    u_mat = A.copy().astype(float)
    l_mat = np.identity(m)
    p_mat = np.identity(m)

    for k in range(m-1):
        # Find pivot
        pivot = np.argmax(np.abs(u_mat[k:, k])) + k

        if pivot != k:
            # Swap rows in u, p, and l
            u_mat[[k, pivot], :] = u_mat[[pivot, k], :]
            p_mat[[k, pivot], :] = p_mat[[pivot, k], :]
            l_mat[[k, pivot], :k] = l_mat[[pivot, k], :k]

        for j in range(k + 1, m):
            l_mat[j, k] = u_mat[j, k] / u_mat[k, k]
            # Subtract multiply of kth row from jth row
            u_mat[j, k:] -= l_mat[j, k] * u_mat[k, k:]

    return p_mat, l_mat, u_mat

def forward_substitution(L, b):
    m, n = L.shape
    x = np.zeros(n)
    for i in range(n):
        x[i] = (b[i] - np.dot(L[i, :i], x[:i])) / L[i, i]
    return x

In [23]:
start_time = time.time()
p, l, u = lu_factorization(A)
y_lu = forward_substitution(l, p @ b)
x_lu = back_substitution(u, y_lu)
print(f"Time taken: {time.time() - start_time} seconds")

print("x difference:", np.linalg.norm(x_lu - x_true))
print("b difference:", np.linalg.norm(b - A @ x_lu))

Time taken: 35.39543128013611 seconds
x difference: 1.310488382211915e-13
b difference: 1.3151040447837372e-13
